In [7]:
from combined_player import player_minutes_value
from html_scraper import db
from mongo_to_db import create_master_df

In [2]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [12]:
pd.set_option('display.max_columns', 50)

# Master Dataframe of All Players

In [29]:
games = db.games.find()
players = db.players.find()

In [30]:
final_df = player_minutes_value(games, players)

In [31]:
final_df.head()

,player_id,player_name,club,birthday,age,foot,squad_num,height,total_minutes_played,transfer_value(sterlings),transfer_value(USD)
0,19132.0,Alexander Domínguez,Squad Club Atlético Vélez Sarsfield,"Jun 5, 1987 (31)",31,right,22.0,"1,95 m",1080.000000,£1.58m,2.01
1,27987.0,Lucas Hoyos,Squad Club Atlético Vélez Sarsfield,"Apr 29, 1989 (29)",29,right,12.0,"1,83 m",90.000000,£135k,0.17
2,30896.0,Gastón Díaz,Squad Club Atlético Vélez Sarsfield,"Mar 13, 1988 (30)",30,right,24.0,"1,75 m",896.483333,£450k,0.57
3,94225.0,Joaquín Laso,Squad Club Atlético Vélez Sarsfield,"Jul 4, 1990 (28)",28,right,6.0,"1,85 m",990.000000,£270k,0.34
4,99779.0,Luis Abram,Squad Club Atlético Vélez Sarsfield,"Feb 27, 1996 (22)",22,left,29.0,"1,80 m",1080.000000,£405k,0.51


# All Shot Data

In [8]:
games = db.games.find()
shots_df = create_master_df(games)

/Users/david/galvanize/super_liga_xg/mongo_to_db.py:69: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  final_df = pd.concat([attach_to_df, df], axis=0, ignore_index=True)
/Users/david/galvanize/super_liga_xg/dataframe_cleaner.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  input_df['is_goal'] = input_df.loc[:, 'shot_type'].isin([9, 11, 12, 13]).astype(int)


In [11]:
shots_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,...,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,...,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1.0
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,...,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763,0,0,0.0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,...,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1.0
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,...,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102,1,0,0.0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,...,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607,0,0,0.0


# Manual Train/Test/Split and Random Forest

In [13]:
from model_prep import create_training_df, create_test_df, create_xG_df, create_summed_xG_df, create_test_min_df, merged_dataframes

In [14]:
train_data, train_y, indices, hold_test = create_training_df(shots_df)
test_data, test_y, indices1, holdout, test = create_test_df(shots_df, hold_test)

In [15]:
print(len(holdout))
#holdout is game_id where as the other two are shot events
print(len(train_data))
len(test_data)

57
1124


740

In [16]:
from sklearn.metrics import log_loss
from sklearn.ensemble import  RandomForestClassifier

In [17]:
random_forest_model = RandomForestClassifier(n_estimators=300, max_depth=3, verbose=1)
random_forest_model.fit(train_data, train_y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.3s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [18]:
p_random_forest = random_forest_model.predict_proba(test_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.1s finished


In [19]:
random_forest_ll = log_loss(test_y, p_random_forest )
random_forest_ll

0.28690652186045973

### Calculating xG and aggregating it for each player

In [20]:
xg_df = create_xG_df(test_data, test_y, p_random_forest)
contributions = create_summed_xG_df(xg_df)

In [21]:
contributions.head()

,player_id,total_xG,total_xA,total_xG+xA,goals
0,55267.0,1.98,0.27,2.25,2.0
1,24256.0,1.94,0.44,2.38,4.0
2,16081.0,1.92,0.72,2.64,1.0
3,77919.0,1.85,0.22,2.07,1.0
4,488.0,1.57,0.34,1.91,2.0


### Calculating Minutes Played for These Games

In [24]:
from mongo_to_db import create_master_player_min_df

In [25]:
games = db.games.find()
players_minutes_df = create_master_player_min_df(games)

In [26]:
pm_df = create_test_min_df(players_minutes_df, test)

### Merge contributions with pm_df

In [28]:
subset_final_df = merged_dataframes(pm_df, contributions)

In [32]:
subset_final_df.head()

,player_name,player_id,total_xG,total_xA,total_xG+xA,goals,xG+xA/90,total_minutes_played
0,Fernando Gaibor,55267.0,1.98,0.27,2.25,2.0,0.600178,337.400000
1,Emmanuel Gigliotti,24256.0,1.94,0.44,2.38,4.0,0.528998,404.916667
2,Luis Rodríguez,16081.0,1.92,0.72,2.64,1.0,0.880000,270.000000
3,Gonzalo Martínez,77919.0,1.85,0.22,2.07,1.0,1.046629,178.000000
4,Lisandro López,488.0,1.57,0.34,1.91,2.0,0.955000,180.000000


### Merge subset_final_df with final_df to provide additional information

In [33]:
final_df.head()

,player_id,player_name,club,birthday,age,foot,squad_num,height,total_minutes_played,transfer_value(sterlings),transfer_value(USD)
0,19132.0,Alexander Domínguez,Squad Club Atlético Vélez Sarsfield,"Jun 5, 1987 (31)",31,right,22.0,"1,95 m",1080.000000,£1.58m,2.01
1,27987.0,Lucas Hoyos,Squad Club Atlético Vélez Sarsfield,"Apr 29, 1989 (29)",29,right,12.0,"1,83 m",90.000000,£135k,0.17
2,30896.0,Gastón Díaz,Squad Club Atlético Vélez Sarsfield,"Mar 13, 1988 (30)",30,right,24.0,"1,75 m",896.483333,£450k,0.57
3,94225.0,Joaquín Laso,Squad Club Atlético Vélez Sarsfield,"Jul 4, 1990 (28)",28,right,6.0,"1,85 m",990.000000,£270k,0.34
4,99779.0,Luis Abram,Squad Club Atlético Vélez Sarsfield,"Feb 27, 1996 (22)",22,left,29.0,"1,80 m",1080.000000,£405k,0.51


In [34]:
columns_to_merge = ['player_id', 'club', 'birthday', 'age', 'foot', 'squad_num', 'transfer_value(USD)']

In [35]:
final_df_part = final_df[columns_to_merge].copy()

In [37]:
subset_final_df.head()

,player_name,player_id,total_xG,total_xA,total_xG+xA,goals,xG+xA/90,total_minutes_played
0,Fernando Gaibor,55267.0,1.98,0.27,2.25,2.0,0.600178,337.400000
1,Emmanuel Gigliotti,24256.0,1.94,0.44,2.38,4.0,0.528998,404.916667
2,Luis Rodríguez,16081.0,1.92,0.72,2.64,1.0,0.880000,270.000000
3,Gonzalo Martínez,77919.0,1.85,0.22,2.07,1.0,1.046629,178.000000
4,Lisandro López,488.0,1.57,0.34,1.91,2.0,0.955000,180.000000


In [36]:
final_df_part.head()

,player_id,club,birthday,age,foot,squad_num,transfer_value(USD)
0,19132.0,Squad Club Atlético Vélez Sarsfield,"Jun 5, 1987 (31)",31,right,22.0,2.01
1,27987.0,Squad Club Atlético Vélez Sarsfield,"Apr 29, 1989 (29)",29,right,12.0,0.17
2,30896.0,Squad Club Atlético Vélez Sarsfield,"Mar 13, 1988 (30)",30,right,24.0,0.57
3,94225.0,Squad Club Atlético Vélez Sarsfield,"Jul 4, 1990 (28)",28,right,6.0,0.34
4,99779.0,Squad Club Atlético Vélez Sarsfield,"Feb 27, 1996 (22)",22,left,29.0,0.51


In [40]:
prediction_df = pd.merge(final_df_part, subset_final_df, on=['player_id'])

In [45]:
prediction_df.head()

,player_id,club,birthday,age,foot,squad_num,transfer_value(USD),player_name,total_xG,total_xA,total_xG+xA,goals,xG+xA/90,total_minutes_played
0,19132.0,Squad Club Atlético Vélez Sarsfield,"Jun 5, 1987 (31)",31,right,22.0,2.01,Alexander Domínguez,NaN,NaN,NaN,NaN,NaN,270.000000
1,27987.0,Squad Club Atlético Vélez Sarsfield,"Apr 29, 1989 (29)",29,right,12.0,0.17,Lucas Hoyos,NaN,NaN,NaN,NaN,NaN,0.000000
2,30896.0,Squad Club Atlético Vélez Sarsfield,"Mar 13, 1988 (30)",30,right,24.0,0.57,Gastón Díaz,0.29,0.16,0.45,0.0,0.229483,176.483333
3,94225.0,Squad Club Atlético Vélez Sarsfield,"Jul 4, 1990 (28)",28,right,6.0,0.34,Joaquín Laso,0.08,0.00,0.08,0.0,0.040000,180.000000
4,99779.0,Squad Club Atlético Vélez Sarsfield,"Feb 27, 1996 (22)",22,left,29.0,0.51,Luis Abram,NaN,NaN,NaN,NaN,NaN,180.000000


In [48]:
columns = ['player_id', 'player_name', 'club', 'birthday', 'age', 'foot', 'squad_num',
       'total_xG', 'total_xA', 'total_xG+xA', 'goals', 'xG+xA/90', 'transfer_value(USD)', 'total_minutes_played']

In [50]:
player_xg_df = prediction_df[columns]

### dataframe with players with xG

In [53]:
pxg = player_xg_df[player_xg_df['total_xG+xA'].notnull()]

In [59]:
pxg[(pxg['age'] < 25) & (pxg['transfer_value(USD)'] < 8) & (pxg['xG+xA/90'] > .3)]

,player_id,player_name,club,birthday,age,foot,squad_num,total_xG,total_xA,total_xG+xA,goals,xG+xA/90,transfer_value(USD),total_minutes_played
140,89000.0,Lautaro Comas,Squad Club Atlético Patronato de la Juventud C...,"Jan 15, 1995 (23)",23,-,7.0,0.41,0.24,0.65,0.0,0.424837,0.17,137.700000
157,135386.0,Leonardo Sequeira,Squad Club Atlético Belgrano,"Apr 26, 1995 (23)",23,right,26.0,1.46,0.18,1.64,1.0,0.328670,1.71,449.083333
159,95766.0,Adrián Balboa,Squad Club Atlético Belgrano,"Jan 19, 1994 (24)",24,right,11.0,0.46,0.29,0.75,0.0,0.396437,0.86,170.266667
190,172801.0,Nahuel Bustos,Squad Club Atlético Talleres,"Jul 4, 1998 (20)",20,right,7.0,0.41,0.29,0.70,0.0,0.350000,0.06,180.000000
256,114108.0,Nicolás Fernández,Squad Club Social y Deportivo Defensa y Justicia,"Feb 8, 1996 (22)",22,right,29.0,0.89,0.47,1.36,0.0,0.486938,3.43,251.366667
272,179441.0,Maximiliano Lovera,Squad Club Atlético Rosario Central,"Mar 9, 1999 (19)",19,right,34.0,0.15,0.08,0.23,0.0,1.713103,2.86,12.083333
284,103961.0,Emanuel Cecchini,Squad Club Atlético Banfield,"Dec 24, 1996 (21)",21,right,5.0,0.14,0.00,0.14,0.0,0.450536,1.71,27.966667
288,193905.0,Julián Carranza,Squad Club Atlético Banfield,"May 22, 2000 (18)",18,right,21.0,0.38,0.16,0.54,1.0,1.080000,2.29,45.000000
337,34042.0,Ricardo Noir,Squad Club Atlético Tucumán,"Mar 12, 1995 (23)",23,right,22.0,0.23,0.00,0.23,0.0,0.655409,0.46,31.583333
394,101036.0,Joel Soñora,Squad Club Atlético Talleres,"Sep 15, 1996 (22)",22,right,21.0,0.07,0.00,0.07,0.0,0.530899,0.17,11.866667
